<a href="https://colab.research.google.com/github/dave502/NLP/blob/main/lesson_16/nlp_hw_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!wget https://www.dropbox.com/s/35dolceuq09w6ur/tokenization_small100.py

In [2]:
%%capture
!pip install transformers[sentencepiece]
!pip install python-telegram-bot
!pip install dropbox
!pip install emoji-country-flag
!pip install pymorphy2
!pip install stop_words
!pip install annoy
!pip install -U sentence-transformers

In [ ]:
!wget https://www.dropbox.com/s/whu54kbx2zpboza/dropbox

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, M2M100ForConditionalGeneration #AutoModelWithLMHead
from tokenization_small100 import SMALL100Tokenizer
from sentence_transformers import SentenceTransformer
import re
import pandas as pd
from tqdm.notebook import tqdm
import dropbox
from pathlib import Path
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import threading
import flag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
import annoy
from gensim.models import Word2Vec, FastText
import numpy as np

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
osm_file_path = 'kaliningrad-latest.osm'
csv_file_path = Path('kaliningrad_nodes.csv')
dropbox_file_path = '/kaliningrad_nodes.csv'
dropbox_file_link = 'https://www.dropbox.com/s/92w8iu94gk0f5py/kaliningrad_nodes.csv'

In [10]:
#@title #####функции загрузки и парсинга osm файла
def upload_dropbox_file(local_file_path, dropbox_file_path):
  with open('/content/dropbox', 'r') as token:
    dropbox_access_token = token.read()
  dropbox_client = dropbox.Dropbox(dropbox_access_token)
  dropbox_client.files_upload(open(local_file_path, "rb").read(), dropbox_file_path, mode=dropbox.files.WriteMode("overwrite"))
  print("[UPLOADED] {}".format(local_file_path))

def download_dropbox_file(local_file_path, dropbox_file_path):
  with open('/content/dropbox', 'r') as token:
    dropbox_access_token = token.read()
  dropbox_client = dropbox.Dropbox(dropbox_access_token)
  try:
    with open(local_file_path, 'wb') as f:
      metadata, result = dropbox_client.files_download(path=dropbox_file_path)
      f.write(result.content)
      print("[DOWNLOADED] {}".format(local_file_path))
      return True
  except Exception as e:
      print('Error downloading file from Dropbox: ' + str(e))
      return False

def osm_parsing_light(osm_file_path: str) -> pd.DataFrame:
  nodes = pd.DataFrame()
  node_data = dict()
  with open(osm_file_path) as osm_data:
    for line in tqdm(osm_data):
      if re.findall(r'(?<=\<node id).*?(?=\/>)', line): # узел без тэгов (<node.../>)
        continue
      elif re.findall(r'(?<=\<node id).*?(?=>)', line): # начался узел с тегами, запоминаем id и координаты (<node id=... ><tag ... /></node>)
        node_attr = re.findall('(id="(.*?)"\s)|(lat="(.*?)"\s)|(lon="(.*?)")', line)
        node_data.update({'id': node_attr[0][1], 'lat': node_attr[1][3], 'lon': node_attr[2][5],})
      elif re.findall(r'(?<=\<tag).*?(?=\/>)', line): # вложенные в узел теги (<tag k="..." v="..."/>), запоминаем другие данные 
        node_tag = re.findall('(k="(.*?)")|(v="(.*?)")', line) # 
        node_data.update({node_tag[0][1]: node_tag[1][3]})
      elif line.strip()== '</node>': # конец узла (</node>)- запись данных и очистка словаря с данными 
        if node_data and node_data.get('name'):
          nodes = nodes.append(node_data, ignore_index=True)
        node_data = dict()
  return nodes     

##Загрузка данных по географическим объектам

In [15]:
if not csv_file_path.is_file():
  csv_downloaded = download_dropbox_file(csv_file_path, dropbox_file_path)

[DOWNLOADED] kaliningrad_nodes.csv


In [16]:
if not csv_downloaded:
  kalingrad_objects_df = osm_parsing_light(osm_file_path)
  kalingrad_objects_df.to_csv(csv_file_path)
  upload_dropbox_file(csv_file_path, dropbox_file_path)

In [17]:
kalingrad_objects_df = pd.read_csv(csv_file_path)

In [18]:
kalingrad_objects_df[kalingrad_objects_df['tourism'].notna()][['id','name', 'lat', 'lon']].head(10)

,id,name,lat,lon
34,71786135,Gołdap (PL) - Гусев (RUS),54.341237,22.297915
110,471830229,Высота Эфа,55.221176,20.906375
114,471928293,Иммануил Кант,54.706492,20.512887
120,480811577,Pension Ehrlich,55.011921,21.243174
141,518447378,Черепаха,54.723725,20.490009
180,550413143,Гостевой дом &quot;Анна&quot;,54.721635,20.419569
197,601372005,След от снаряда на ферме моста,54.706364,20.489163
200,601374809,Борющиеся зубры,54.720445,20.497014
201,601374994,Памятный знак в виде верстового столба к 750-л...,54.718523,20.493949
243,679289752,Бабушка хомлин Марта,54.721984,20.523246


In [19]:
kalingrad_objects_df.columns

Index(['Unnamed: 0', 'id', 'lat', 'lon', 'addr:city', 'addr:country',
       'addr:region', 'admin_level', 'capital', 'int_name',
       ...
       'contact:viber', 'contact:whatsapp', 'service:bicycle:parts',
       'service:bicycle:repair', 'wreck:date_sunk', 'wreck:type',
       'wreck:visible_at_high_tide', 'wreck:visible_at_low_tide', 'club',
       'owner'],
      dtype='object', length=520)

##Предобработка данных

Функция лемматизации и фильтрции текста

In [20]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_text(line):
  txt = ''.join(i for i in line.strip() if i not in exclude).split()
  txt = [morpher.parse(i.lower())[0].normal_form for i in txt]
  txt = [i for i in txt if i not in sw and i != '']
  return txt

Выбор из полученного датафрейма только нужных колонок

In [21]:
# main information
columns_with_name = ['name']
column_id = ['id']
columns_with_position = list(kalingrad_objects_df.columns[kalingrad_objects_df.columns.str.contains(r"lat|lon")])
# columns_with_name = list(kalingrad_objects_df.columns[kalingrad_objects_df.columns.str.contains(r"name:")])
columns_with_description = list(kalingrad_objects_df.columns[kalingrad_objects_df.columns.str.contains(r"description")])
columns_with_amenity = list(kalingrad_objects_df.columns[kalingrad_objects_df.columns.str.contains(r"amenity")])
columns_with_tourism = list(kalingrad_objects_df.columns[kalingrad_objects_df.columns.str.contains(r"historic|tourism")])
# additional informaton

columns_with_opening_hours = list(kalingrad_objects_df.columns[kalingrad_objects_df.columns.str.contains(r"opening_hours")])
columns_with_phone = list(kalingrad_objects_df.columns[kalingrad_objects_df.columns.str.contains(r"phone")])
columns_with_website = list(kalingrad_objects_df.columns[kalingrad_objects_df.columns.str.contains(r"website")])
# get new DF with useful columns
clusterizated_columns = sum([columns_with_name, columns_with_description, columns_with_amenity, columns_with_tourism], [])
kalingrad_df = kalingrad_objects_df[sum([column_id, columns_with_position, clusterizated_columns], [])]
kalingrad_df.fillna('', inplace=True)
kalingrad_df['text'] = kalingrad_df[clusterizated_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
kalingrad_df['text'] = kalingrad_df['text'].apply(lambda x: preprocess_text(str(x)))

In [23]:
kalingrad_df[kalingrad_df['tourism']!=''].head(3)

,id,lat,lon,population,population:date,platforms,seamark:buoy_lateral:colour,name,description,description:de,description:en,description:ru,amenity,disused:amenity,was:amenity,historic,tourism,historic:civilization,construction:tourism,text
34,71786135,54.341237,22.297915,,,,,Gołdap (PL) - Гусев (RUS),,,,,,,,,information,,,"[gołdap, pl, гусев, rus, information]"
110,471830229,55.221176,20.906375,,,,,Высота Эфа,,,,,,,,,viewpoint,,,"[высота, эфа, viewpoint]"
114,471928293,54.706492,20.512887,,,,,Иммануил Кант,,,,,,,,tomb,attraction,,,"[иммануил, кант, tomb, attraction]"


##Кластеризация географических объектов

In [24]:
tfidf_vect = TfidfVectorizer().fit(" ".join(txt) for txt in kalingrad_df['text'].values)
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [25]:
modelFT = FastText(sentences=kalingrad_df['text'], size=100, min_count=1, window=5)
ft_index = annoy.AnnoyIndex(100 ,'angular')
midf = np.mean(tfidf_vect.idf_)
index_map = {}
counter = 0

for i in tqdm(range(len(kalingrad_df))):
    n_ft = 0
    index_map[counter] = (kalingrad_df.loc[i, "name"], kalingrad_df.loc[i, "lat"],\
                          kalingrad_df.loc[i, "lon"], kalingrad_df.loc[i, "description"])
    vector_ft = np.zeros(100)
    for word in kalingrad_df.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index.add_item(counter, vector_ft)
    counter += 1

ft_index.build(1000)


  0%|          | 0/10317 [00:00<?, ?it/s]

True

Функция преобразования текста в эмбеддинг

In [26]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    txt = preprocess_text(txt)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [27]:
# kalingrad_df.query('name.str.contains("кафе")', engine='python')

In [28]:
# model = SentenceTransformer('xlm-roberta-base')

# embeddings = model.encode(kalingrad_df['text'].apply(lambda row: ' '.join(row)), show_progress_bar=True)
# kalingrad_df['Emb']=""
# kalingrad_df['Emb'] = kalingrad_df['Emb'].astype(object)
# for i, emb in enumerate(embeddings):
#   kalingrad_df.at[i, 'Emb'] = emb

# sent = "свободы"
# sent_tens = model.encode(sent, convert_to_tensor='pt').to(device)
# results = kalingrad_df['Emb'].apply(lambda emb: torch.cosine_similarity(sent_tens, torch.FloatTensor(emb).to(device), 0).item())

##Инициализация модели болталки

In [43]:
# model for chat
model_name_chat = 'sberbank-ai/rugpt3medium_based_on_gpt2' #rugpt3small_based_on_gpt2
tokenizer_chat = AutoTokenizer.from_pretrained(model_name_chat)
model_chat = AutoModelForCausalLM.from_pretrained(model_name_chat).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


История всего чата хранится в переменной dialog_text

In [45]:
user_label, bot_label = '\n@user:', '\n@bot:'
dialog_text = '' # chat history

def chatterbox(look_behind_depth=20):
  """returns response for text from dialog_text variable
     and writes the generated text back in dialog_text 
     look_behind_depth - how many phrases to take for generation"""

  global dialog_text, user_label, bot_label
  # перед новым сообщением бота добавляется метка бота
  dialog_text += bot_label
  
  # находим позицию последнего n-го сообщения (параметр look_behind_depth)
  result = [_.start() for _ in re.finditer(user_label, dialog_text)] 
  start = result[0 if len(result)<look_behind_depth else (look_behind_depth-1)]

  # длина обрабатываемого диалога, понадобится для отсечения сгенерированного текста в дальнейшем
  len_without_answer = len(dialog_text[start:])

  # токенизация последних n сообщений
  input_tokens = tokenizer_chat(dialog_text[start:], return_tensors='pt')
  input_tokens = {k: v.to(model_chat.device) for k,v in input_tokens.items()}
  end_token_id = tokenizer_chat.encode('\n')[0]

  # генерация следующей фразы
  size = input_tokens['input_ids'].shape[1]
  output = model_chat.generate(
      **input_tokens,
      eos_token_id=end_token_id,
      do_sample=True,
      max_length=size+128,
      repetition_penalty=3.2,
      temperature=1,
      num_beams=3,
      length_penalty=0.01,
      pad_token_id=tokenizer_chat.eos_token_id     
  )
  # из эмбединга - в текст
  output = tokenizer_chat.decode(output[0])
  
  # удаление лишнего текста, остаётся только сгенерированная фраза
  response = output[len_without_answer:].strip()

  # add next phrase to general dialog text
  dialog_text += response

  return response


In [31]:
# dialog_text = user_label + input('Начните диалог:\n') # получаем первую фразу от пользователя
# while True:
#   response = chatterbox()   # получаем ответ от модели на основании последних 10-ти фраз
#   next_sentence = input(response + '\n') # выводим ответ и получаем следующую фразу от пользователя
#   if next_sentence.lower()=='пока': break # если пользователь прощается - выходим
#   dialog_text += user_label + next_sentence  # запись новых фраз в историю диалога

##Инициализация модели для мультиязычного перевода

In [44]:
# model for translation
model_tr = M2M100ForConditionalGeneration.from_pretrained("alirezamsh/small100")
tokenizer_tr = SMALL100Tokenizer.from_pretrained("alirezamsh/small100")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'M2M100Tokenizer'. 
The class this function is called from is 'SMALL100Tokenizer'.


In [33]:
# from transformers import pipeline
# translator = pipeline("translation_en_to_de")
# print(translator("Hugging Face is a technology company based in New York and Paris", max_length=40))

##Бот

<font color='red' > <b>
Сообщения пользователя делятся на 3 типа: <br>
- сообщения с запросом на перевод текста - заканчиваются двумя символами, определяющими язык перевода (напр.: как пройти на красную площадь? es)   <br>
- сообщения с запросом поиска туристических объектов - начинаются со слова "где" (напр.: где кинотеатр?)  <br>
- простые сообщения в чат в целях общения с ботом - все остальные
</b></font>

Классификации сообщений пользователя моделью, как на лекции, нет, т.к. не использовались датасеты с вопросами

In [35]:
# загрузка токена для telegram бота
telegram_token_file = Path('telegram_token')
download_dropbox_file(telegram_token_file, '/travelot_bot')
telegram_token = telegram_token_file.read_text()

[DOWNLOADED] telegram_token


In [51]:
# список языков для перевода (модель поддерживает больше, для примера взяты несколько наугад)
languages = ['en', 'es', 'de', 'ru', 'gb']
flags = {'en':'gb'} # словарь для отображения флажка языка для наглядности
questions = ['где'] # вопросы для передачи управления функции поиска геообъектов 

# на старте только приветствие
def start(update: Update, context: CallbackContext):
  update.message.reply_text('Привет!')
# команда /stop останавливает выполнение Updater'а
def stop(update: Update, context: CallbackContext):
   threading.Thread(target=shutdown).start()

# ответ пользователю в зависимости от запроса
def echo(update: Update, context: CallbackContext):
  global dialog_text
  
  txt = update.message.text
  # перевод если последние 2 символа есть в списке [languages]
  if (to_language:=txt.rstrip().split()[-1]) in languages:
    # рисунок флага для наглядности
    flag_icon = flag.flag(to_language if to_language not in flags.keys() else flags.get(to_language))
    #ответ
    update.message.reply_text(flag_icon + ' \n' + \
      translate(update.message.text[:-2], to_language)[0])
  # вывод координат искомого объекта, если вопрос начинается с "где" 
  # выводится один объект без привязки к нахождению пользователя, т.е. информация на текущий момент малополезная 
  elif (question:=txt.split()[0]) in questions:
    vect_ft = embed_txt(update.message.text[len(question):], idfs, midf)
    ft_index_val = ft_index.get_nns_by_vector(vect_ft, 5)
    results = []
    for item in ft_index_val:
      results.append(index_map[item])
    # выводится только первый найденный объект
    context.bot.send_location(chat_id=update.message.chat_id, latitude=results[0][1], longitude=results[0][2])
    update.message.reply_text(f'{results[0][0]}\nКоординаты: {results[0][1]} {results[0][2]}\n{results[0][3]}')
  # иначе отвечает болталка
  else:
    dialog_text += user_label + update.message.text
    response = chatterbox()
    update.message.reply_text(response)

def shutdown():
    updater.stop()
    updater.is_idle = False

def translate(text, to_language):
  tokenizer_tr.tgt_lang = to_language
  encoded_hi = tokenizer_tr(text, return_tensors="pt")
  generated_tokens = model_tr.generate(**encoded_hi)
  translated_text = tokenizer_tr.batch_decode(generated_tokens, skip_special_tokens=True)
  return translated_text

In [52]:
updater = Updater(telegram_token, use_context=True)
dispatcher = updater.dispatcher

dispatcher.add_handler(CommandHandler('start', start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
dispatcher.add_handler(CommandHandler('stop', stop))

updater.start_polling()
updater.idle()